# Creating Schemas & Tables with Managed Storage in Unity Catalog

**Objective:** Understand the hierarchy of data storage in Unity Catalog by creating **Schemas** (Databases) and **Tables** in three different scenarios to see where the data physically lands.

## Scenario Overview
We will create three Schemas and Tables:

1.  **Default Schema:** In a standard Catalog (uses Metastore Root Storage).
2.  **External Catalog Schema:** In a Catalog that has its own External Location (uses Catalog Storage).
3.  **External Schema:** A Schema with its own External Location (overrides Catalog storage).

---

## 1. Scenario 1: Default Storage Behavior
We will use the `dev` catalog (created previously) which has no specific location. It falls back to the Metastore root.

In [ ]:
-- 1. Create Schema in 'dev' catalog
CREATE SCHEMA IF NOT EXISTS dev.bronze
COMMENT 'This schema uses the default Metastore storage';

-- 2. Create a Managed Table
CREATE TABLE IF NOT EXISTS dev.bronze.raw_sales (
    id INT,
    amount DOUBLE
);

-- 3. Insert Dummy Data
INSERT INTO dev.bronze.raw_sales VALUES (1, 100.50), (2, 200.75);

-- 4. Verify Location
-- Look for 'Location'. It should start with the Metastore root path (e.g., abfss://root@...)
DESCRIBE EXTENDED dev.bronze.raw_sales;

## 2. Scenario 2: Catalog-Level Storage Isolation
We will use the `dev_ext` catalog (created previously) which has a specific External Location defined.

In [ ]:
-- 1. Create Schema in 'dev_ext' catalog
-- We do NOT specify a location here. It should inherit from the Catalog.
CREATE SCHEMA IF NOT EXISTS dev_ext.bronze
COMMENT 'This schema inherits storage from dev_ext catalog';

-- 2. Create a Managed Table
CREATE TABLE IF NOT EXISTS dev_ext.bronze.raw_sales (
    id INT,
    amount DOUBLE
);

-- 3. Insert Data
INSERT INTO dev_ext.bronze.raw_sales VALUES (10, 500.00);

-- 4. Verify Location
-- Look for 'Location'. It should point to the path defined for the 'dev_ext' catalog.
DESCRIBE EXTENDED dev_ext.bronze.raw_sales;

## 3. Scenario 3: Schema-Level Storage Isolation
We will create a new schema inside `dev_ext` but override the location to point to a specific folder. This is useful if a specific dataset needs to be in a different container/path than the rest of the catalog.

### Prerequisite: Create External Location for Schema
We need a specific path authorized.

In [ ]:
-- 1. Define the external location for the schema
-- Format: abfss://<container>@<storage>.dfs.core.windows.net/<path>/schema_ext
-- Note: We reuse the same Storage Credential
CREATE EXTERNAL LOCATION IF NOT EXISTS ext_schema_loc
    URL 'abfss://data@adbself01.dfs.core.windows.net/adb/schema_ext'
    WITH STORAGE CREDENTIAL sc_catalog_storage;

-- 2. Create Schema with MANAGED LOCATION
CREATE SCHEMA IF NOT EXISTS dev_ext.bronze_ext
    MANAGED LOCATION 'abfss://data@adbself01.dfs.core.windows.net/adb/schema_ext';

-- 3. Create a Managed Table
CREATE TABLE IF NOT EXISTS dev_ext.bronze_ext.raw_sales (
    id INT,
    amount DOUBLE
);

-- 4. Insert Data
INSERT INTO dev_ext.bronze_ext.raw_sales VALUES (99, 999.99);

-- 5. Verify Location
-- It should point to the specific schema path, ignoring the catalog's default path.
DESCRIBE EXTENDED dev_ext.bronze_ext.raw_sales;

## 4. Summary of Storage Precedence

When you create a **Managed Table**, Unity Catalog determines the storage location in this order:

1.  **Schema Level:** Is `MANAGED LOCATION` defined on the Schema? -> **Yes? Use it.**
2.  **Catalog Level:** Is `MANAGED LOCATION` defined on the Catalog? -> **Yes? Use it.**
3.  **Metastore Level:** **Fallback** to the Metastore's root storage.

## Next Steps
In the next video, we will explore **External Tables** in Unity Catalog, how they differ from Managed Tables, and perform operations like `DROP TABLE` to see the impact on underlying data.